In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [12]:
# Load the data
nordpool = pd.read_csv('New Data/2021/Nordpool DKK.csv')
energinet = pd.read_csv('New Data/2021/Energinet DKK.csv')
weather_observation = pd.read_csv('New Data/2021/Weather Observation.csv')
weather_forecast = pd.read_csv('New Data/2021/Weather Forecast.csv')
network = pd.read_csv('New Data/2021/Network Manager.csv')

# Filter energinet price area
energinet = energinet[energinet['PriceArea | PriceArea | 804696'] == 'DK2']
# Drop price area
energinet.drop('PriceArea | PriceArea | 804696', axis=1, inplace=True)

# Convert ts to datetime
nordpool['ts'] = pd.to_datetime(nordpool['ts'])
energinet['ts'] = pd.to_datetime(energinet['ts'])
weather_observation['ts'] = pd.to_datetime(weather_observation['ts'])
weather_forecast['ts'] = pd.to_datetime(weather_forecast['ts'])
network['ts'] = pd.to_datetime(network['ts'])

# Resample network_manager data to hourly intervals (average the data)
network = network.set_index('ts')
network = network.resample('H').mean()
network.reset_index(inplace=True)

/var/folders/tz/wvdldxqj00gc2cslzkpx77sr0000gn/T/ipykernel_59592/2096482961.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  network = network.resample('H').mean()


#### Data Handling

In [13]:
# Compare length of dfs + get first & last timestamp
def get_first_last(**dfs):
    # Initialize
    first_last = pd.DataFrame(index=['first', 'last', 'length', 'start_check', 'end_check'])
    col_name='ts'
    # Save first and last timestamp for each dataframe
    for name, df in dfs.items():
        first_ts = df['ts'].iloc[0]
        last_ts = df['ts'].iloc[-1]
        start_check = start == first_ts
        end_check = end == last_ts
        first_last[name] = [first_ts, last_ts, len(df), start_check, end_check]
    return first_last

# Initialize
start = pd.to_datetime('2021-01-01 00:00:00')
end = pd.to_datetime('2021-12-31 23:00:00')

print("Length of a year in hours:", 365*24)
get_first_last(nordpool=nordpool, energine=energinet, weather_observation=weather_observation, weather_forecast=weather_forecast, network=network)

Length of a year in hours: 8760


,nordpool,energine,weather_observation,weather_forecast,network
first,2021-01-02 00:00:00,2021-01-01 00:00:00,2021-01-15 16:00:00,2021-01-01 00:00:00,2021-01-01 00:00:00
last,2021-12-31 23:00:00,2021-12-31 23:00:00,2021-12-31 23:00:00,2021-12-29 11:00:00,2021-12-31 23:00:00
length,8735,8759,7936,8694,8760
start_check,False,True,False,True,True
end_check,True,True,True,False,True


In [14]:
# Exclude all before 15/01/2022 for all dataframes
start = pd.to_datetime('2021-01-15 00:00:00')
nordpool = nordpool[nordpool['ts'] >= start]
nordpool.reset_index(drop=True, inplace=True)
energinet = energinet[energinet['ts'] >= start]
energinet.reset_index(drop=True, inplace=True)
weather_observation = weather_observation[weather_observation['ts'] >= start]
weather_observation.reset_index(drop=True, inplace=True)
weather_forecast = weather_forecast[weather_forecast['ts'] >= start]
weather_forecast.reset_index(drop=True, inplace=True)
network = network[network['ts'] >= start]
network.reset_index(drop=True, inplace=True)

In [16]:
get_first_last(nordpool=nordpool, energinet=energinet, weather_observation=weather_observation, weather_forecast=weather_forecast, network=network)

,nordpool,energinet,weather_observation,weather_forecast,network
first,2021-01-15 00:00:00,2021-01-15 00:00:00,2021-01-15 16:00:00,2021-01-15 00:00:00,2021-01-15 00:00:00
last,2021-12-31 23:00:00,2021-12-31 23:00:00,2021-12-31 23:00:00,2021-12-29 11:00:00,2021-12-31 23:00:00
length,8423,8423,7936,8358,8424
start_check,True,True,False,True,True
end_check,True,True,True,False,True


In [17]:
# Merge the dataframes on network with left join
data = pd.merge(network, weather_observation, on='ts', how='left')
data = pd.merge(data, weather_forecast, on='ts', how='left')

# Sort the data by the timestamp column 'ts' to ensure time order
data = data.sort_values(by='ts')

data.head()

,ts,Hasle Common 10kV Lines Voltage | has_fel_10kvskinnespend | 804133,Hasle Vind Active Power | has_vin_effekt | 804123,Hasle Vind Current | has_vin_belastning | 804117,Observed maximum temperature past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/temp_max_past1h/06193 | 406592,Observed mean intensity of global radiation in the latest hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/radia_glob_past1h/06193 | 406672,Observed minimum temperature past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/temp_min_past1h/06193 | 406608,Observed mean wind speed the past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/wind_speed_past1h/06193 | 406640,Observed mean humidity past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/humidity_past1h/06193 | 406576,Observed mean temperature past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/temp_mean_past1h/06193 | 406560,...,Weather forecast for the max air temperature for the coming 6 hours at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_max | 128206,Weather forecast for wind direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/wind_direction | 128270,Weather forecast for the precipitation amount for the coming 1 hour at Bornholm | 9F7P/00/00/MET-Norway/forecast/precipitation_amount_acc | 128238,Weather forecast for the relativehumidity amount at Bornholm | 9F7P/00/00/MET-Norway/forecast/relative_humidity_2m | 128254,Weather forecast for wind speed at Bornholm | 9F7P/00/00/MET-Norway/forecast/wind_speed | 128286,Weather forecast for wind speed y_direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/y_wind_10m | 128318,Weather forecast for solar shortwave flux at Bornholm | 9F7P/00/00/MET-Norway/forecast/integral_of_surface_downwelling_shortwave_flux_in_air_wrt_time | 128334,Weather forecast for the minimum air temperature for the coming 6 hours at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_min | 128222,Weather forecast for wind speed x_direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/x_wind_10m | 128302,Weather forecast for the air temperature at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_2m | 128190
0,2021-01-15 00:00:00,10.431320,0.975518,58.290779,NaN,NaN,NaN,NaN,NaN,NaN,...,272.20663,15.992242,0.001953,0.820623,3.853889,-3.686989,0.000000e+00,272.20834,-0.681063,272.51380
1,2021-01-15 01:00:00,10.471132,1.399003,77.685317,NaN,NaN,NaN,NaN,NaN,NaN,...,272.25000,8.867309,4.557617,0.815664,3.774305,-3.492050,2.621432e-10,272.29065,-0.240537,272.10806
2,2021-01-15 02:00:00,10.440726,1.340842,77.675711,NaN,NaN,NaN,NaN,NaN,NaN,...,271.53186,358.140870,1.428711,0.830938,3.622615,-3.561020,3.277713e-10,270.79755,-0.678896,271.12054
3,2021-01-15 03:00:00,10.410453,1.400077,79.632210,NaN,NaN,NaN,NaN,NaN,NaN,...,271.45258,21.992271,3.289062,0.860692,4.625701,-3.617756,3.934847e-10,272.06714,-1.737806,272.08910
4,2021-01-15 04:00:00,10.471521,1.499689,84.263752,NaN,NaN,NaN,NaN,NaN,NaN,...,271.49908,10.687552,0.583740,0.848837,4.578940,-4.491100,0.000000e+00,271.71410,-1.258742,271.16480


In [19]:
# Column cleaning

# Use regex to remove everything starting from '|'
data.columns = data.columns.str.replace(r'\|.*', '', regex=True)
nordpool.columns = nordpool.columns.str.replace(r'\|.*', '', regex=True)
energinet.columns = energinet.columns.str.replace(r'\|.*', '', regex=True)

# Use regex to remove everything starting from the word "at"
data.columns = data.columns.str.replace(r'\sat.*', '', regex=True)

# Strip columns
data.columns = data.columns.str.strip()

data.columns

Index(['ts', 'Hasle Common 10kV Lines Voltage', 'Hasle Vind Active Power',
       'Hasle Vind Current', 'Observed maximum temperature past hour',
       'Observed mean intensity of global radiation in the latest hour',
       'Observed minimum temperature past hour',
       'Observed mean wind speed the past hour',
       'Observed mean humidity past hour',
       'Observed mean temperature past hour',
       'Observed mean wind direction the past hour',
       'Observed accumulated precipitation the past hour',
       'Weather forecast for the max air temperature for the coming 6 hours',
       'Weather forecast for wind direction',
       'Weather forecast for the precipitation amount for the coming 1 hour',
       'Weather forecast for the relativehumidity amount',
       'Weather forecast for wind speed',
       'Weather forecast for wind speed y_direction',
       'Weather forecast for solar shortwave flux',
       'Weather forecast for the minimum air temperature for the coming 6

In [20]:
data.isnull().sum()

ts                                                                           0
Hasle Common 10kV Lines Voltage                                            594
Hasle Vind Active Power                                                    594
Hasle Vind Current                                                         594
Observed maximum temperature past hour                                     492
Observed mean intensity of global radiation in the latest hour             491
Observed minimum temperature past hour                                     492
Observed mean wind speed the past hour                                     492
Observed mean humidity past hour                                           489
Observed mean temperature past hour                                        492
Observed mean wind direction the past hour                                 491
Observed accumulated precipitation the past hour                           490
Weather forecast for the max air temperature for the